In [11]:
# ! pip install -U deepeval
# ! pip install numpy
# ! pip install transformers
# ! pip3 install torch torchvision torchaudio

In [1]:
# ! pip install sentencepiece
# ! pip install tokenizers
# ! pip install accelerate
# ! pip install bitsandbytes
# ! pip install vllm
# ! pip install fraction
# ! pip install protobuf

In [1]:
from deepeval.benchmarks import MMLU, TruthfulQA
from deepeval.benchmarks.tasks import MMLUTask, TruthfulQATask
from deepeval.benchmarks.modes import TruthfulQAMode
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM
from typing import List
import torch

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {torch_device}")

/zhome/25/7/186664/dl/lib64/python3.9/site-packages/deepeval/__init__.py:53: UserWarning: You are using deepeval version 1.5.9, however version 2.0 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


Using device: cuda


In [2]:
blackhole_dir = #blackhole dirdd

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", cache_dir=blackhole_dir)
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", cache_dir=blackhole_dir)

model = AutoModelForCausalLM.from_pretrained("meta-math/MetaMath-Mistral-7B", cache_dir=blackhole_dir)
tokenizer = AutoTokenizer.from_pretrained("meta-math/MetaMath-Mistral-7B", cache_dir=blackhole_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [97]:
class Mistral7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto 

        # prompt = "[INST] " + prompt + " [/INST]"
        prompt = prompt + "\nYou will only respond with a JSON object with the keys Explanation and Answer Choice, where Explanation is a string and Answer Choice is either 'A', 'B', 'C' or 'D'."

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, pad_token_id=self.tokenizer.eos_token_id)#, temperature=0.0001)
        decoded_tokens = self.tokenizer.batch_decode(generated_ids)[0]
        # return decoded_tokens[len(prompt)-50:]
        return decoded_tokens
        return decoded_tokens[len(prompt):]
        return decoded_tokens[len(prompt)+5:]
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    # This is optional.
    def batch_generate(self, promtps: List[str]) -> List[str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer(promtps, return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)

    def get_model_name(self):
        return "Mistral 7B"

mistral_7b = Mistral7B(model=model, tokenizer=tokenizer)
# print(mistral_7b.generate("Write me a joke"))

In [98]:
print(mistral_7b.generate("Hello! What is 5 minus 2?\nA. 3\nB. 5\nC. 25\nD. 50\nAnswer: \n\nOutput only 'A', 'B', 'C', or 'D'."))

<s> Hello! What is 5 minus 2?
A. 3
B. 5
C. 25
D. 50
Answer: 

Output only 'A', 'B', 'C', or 'D'.
You will only respond with a JSON object with the keys Explanation and Answer Choice, where Explanation is a string and Answer Choice is either 'A', 'B', 'C' or 'D'.
The answer is: </s>


In [99]:
print(mistral_7b.generate("The following are multiple choice questions (with answers) about high school biology.\n\nWhich of the following is not a way to form recombinant DNA?\nA. Translation\nB. Conjugation\nC. Specialized transduction\nD. Transformation\nAnswer: "))

<s> The following are multiple choice questions (with answers) about high school biology.

Which of the following is not a way to form recombinant DNA?
A. Translation
B. Conjugation
C. Specialized transduction
D. Transformation
Answer: 
You will only respond with a JSON object with the keys Explanation and Answer Choice, where Explanation is a string and Answer Choice is either 'A', 'B', 'C' or 'D'.

The answer is: D</s>


In [100]:
print(mistral_7b.generate("If a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\nA. (0, – 3)\nB. (4, 1)\nC. (2, 2)\nD. (– 4, –2)\nAnswer:"))

<s> If a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is
A. (0, – 3)
B. (4, 1)
C. (2, 2)
D. (– 4, –2)
Answer:
You will only respond with a JSON object with the keys Explanation and Answer Choice, where Explanation is a string and Answer Choice is either 'A', 'B', 'C' or 'D'.  The answer is: A</s>


In [15]:
mm_tasks = [MMLUTask.HIGH_SCHOOL_MATHEMATICS] #, MMLUTask.HIGH_SCHOOL_BIOLOGY]
tqa_tasks = [TruthfulQATask.SCIENCE]
tqa_mode = TruthfulQAMode.MC1 # Use MC1 as a benchmark for pinpoint accuracy and MC2 for depth of understanding.

# Define benchmark with specific tasks and shots
benchmark = MMLU(
    tasks=mm_tasks,
    n_shots=3
)

TQABenchmark = TruthfulQA(
    tasks=tqa_tasks,
    mode=tqa_mode
)

In [16]:
# Replace 'mistral_7b' with your own custom model
benchmark.evaluate(model=mistral_7b)
results = benchmark.predictions
print(benchmark.overall_score)

Processing high_school_mathematics: 100%|█████| 270/270 [02:23<00:00,  1.88it/s]

MMLU Task Accuracy (task=high_school_mathematics): 0.0
Overall MMLU Accuracy: 0.0
0.0


In [17]:
for i in range(20):
    print(f'Input {i+1}:')
    print('=**********=')
    print(benchmark.predictions['Input'][i])
    print('=**********=')
    print(f'Prediction {i+1}:')
    print('++++++++++')
    # print(results['Prediction'][i][results['Prediction'][i].find("\n\nOutput 'A', 'B', 'C', or 'D'. Full answer not needed.")+55:])
    print(results['Prediction'][i])
    print('-'*50)

Input 1:
=**********=
If a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is
A. (0, – 3)
B. (4, 1)
C. (2, 2)
D. (– 4, –2)
Answer:
=**********=
Prediction 1:
++++++++++
NST]</s>
--------------------------------------------------
Input 2:
=**********=
The length of a rectangle is twice its width. Given the length of the diagonal is $5\sqrt{5}$, find the area of the rectangle.
A. 2500
B. 2
C. 50
D. 25
Answer:
=**********=
Prediction 2:
++++++++++
NST]</s>
--------------------------------------------------
Input 3:
=**********=
A positive integer n is called “powerful” if, for every prime factor p of n, p^2 is also a factor of n. An example of a powerful number is
A. 392
B. 336
C. 300
D. 297
Answer:
=**********=
Prediction 3:
++++++++++
NST]</s>
--------------------------------------------------
Input 4:
=**********=
At breakfast, lunch, and dinner, Joe randomly choos

In [87]:
results['Input'][0]

'If a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\nA. (0, – 3)\nB. (4, 1)\nC. (2, 2)\nD. (– 4, –2)\nAnswer:'

In [31]:
print(results['Prediction'][0])

<s> [INST] The following are multiple choice questions (with answers) about high school mathematics.

Joe was in charge of lights for a dance. The red light blinks every two seconds, the yellow light every three seconds, and the blue light every five seconds. If we include the very beginning and very end of the dance, how many times during a seven minute dance will all the lights come on at the same time? (Assume that all three lights blink simultaneously at the very beginning of the dance.)
A. 3
B. 15
C. 6
D. 5
Answer: B

Five thousand dollars compounded annually at an $x\%$ interest rate takes six years to double. At the same interest rate, how many years will it take $\$300$ to grow to $\$9600$?
A. 12
B. 1
C. 30
D. 5
Answer: C

The variable $x$ varies directly as the square of $y$, and $y$ varies directly as the cube of $z$. If $x$ equals $-16$ when $z$ equals 2, what is the value of $x$ when $z$ equals $\frac{1}{2}$?
A. -1
B. 16
C. -\frac{1}{256}
D. \frac{1}{16}
Answer: C

If a pen

In [57]:
# benchmark.predictions['Prediction'][0].count('\nAnswer:')
benchmark.predictions['Prediction'][0] # [benchmark.predictions['Prediction'][0].find('\nAnswer:')+9:]

''

In [29]:
# results = benchmark.evaluate(model=mistral_7b, batch_size=5)
# print("Overall Score: ", results)